# Anticipez les besoins en consommation électrique de bâtiments

- **Projet 4 du parcours « Data Scientist » d’OpenClassrooms**
- **Mark Creasey**

## Partie 1 : Nettoyage et analyse exploratoire des données

<img width="200" src="https://user.oc-static.com/upload/2019/02/24/15510245026714_Seattle_logo_landscape_blue-black.png" alt="Logo seattle">


# 1. Compréhension du problème

## 1.1 Mission

À partir des [relevés déjà réalisés en 2015 et 2016](https://www.kaggle.com/city-of-seattle/sea-building-energy-benchmarking#2015-building-energy-benchmarking.csv) :

- **prédire les émissions de CO2** et la **consommation totale d’énergie** de bâtiments commerciales en Seattle pour lesquels elles n’ont pas encore été mesurées, basé sur les données déclaratives du permis d'exploitation commerciale (taille et usage des bâtiments, mention de travaux récents, date de construction…).

- **évaluer l’intérêt de l’[ENERGY STAR Score](https://www.energystar.gov/buildings/facility-owners-and-managers/existing-buildings/use-portfolio-manager/interpret-your-results/what) pour la prédiction d’émissions**.


### 1.1.1 Quelles variables pour prédire les consommations / emissions CO2 ?

Il faut prédire les targets uniquement avec les données disponibles sur le permis d'exploitation commerciale :

- on ne peut pas utiliser les relevés de consommation d'électricité, de gaz, etc comme paramètres des prévisions.
- on peut utiliser type d'usage, surface, nombre d'étages, moyenne de chauffage, ...


### 1.1.2 Quels bâtiments à inclure dans les prévisions ?

Il faut prédire les targets uniquement pour les bâtiments commerciaux.

Donc, il faut éliminer les bâtiments residential, si possible.


## 1.2 Requirements : Bibliothèques utilisées dans ce notebook

- voir [`requirements.txt`](./requirements.txt) pour les versions des bibliothèques testées avec ce notebook

### 1.2.1 Import des bibliothèques


In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy


### 1.2.2 Liste des versions des bibliothèques utilisées


In [2]:
from platform import python_version

python_version()
print('versions des bibliothèques utilisées:')
print('; '.join(f'{m.__name__}=={m.__version__}' for m in globals(
).values() if getattr(m, '__version__', None)))


versions des bibliothèques utilisées:
json==2.0.9; numpy==1.21.5; pandas==1.1.5; seaborn==0.11.2; scipy==1.7.3


### 1.2.3 Configuration défauts d'affichage


In [3]:
pd.set_option('display.max_columns', 200)  # pour afficher toutes les colonnes
pd.set_option('display.max_rows', 10)  # pour afficher max 10 lignes
pd.set_option('display.max_colwidth', 800)  # pour afficher toutes la text
# pd.set_option('display.precision', 2)

%matplotlib inline
sns.set_theme(style="white", context="notebook")
sns.set_color_codes("pastel")
sns.set_palette("tab20")


## 1.3 Des fonctions utilitaires

### 1.3.1 Enregistrement des graphiques

Pour enregistrer les graphiques, define **`SAVE_IMAGES = True`**


In [4]:
SAVE_IMAGES = True
IMAGE_FOLDER = './images'
if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)


def to_png(fig_name=None):
    """
    Enregistre l'image dans un fichier,
    il faut appeler avant plt.show() pour pouvoir ajuster la taille de l'image
    avec bbox_inches=tight pour être sûr d'inclure le titre / legend entier.
    """

    def get_title():
        if plt.gcf()._suptitle is None:  # noqa
            return plt.gca().get_title()
        else:
            return plt.gcf()._suptitle.get_text()  # noqa

    if SAVE_IMAGES:
        if fig_name is None:
            fig_name = get_title()
        elif len(fig_name) < 9:
            fig_name = f'{fig_name}_{get_title()}'
        fig_name = fig_name.replace(' ', '_').replace(
            ':', '-').replace('.', '-').replace('/', '_')
        print(f'"{fig_name}.png"')
        plt.gcf().savefig(
            f'{IMAGE_FOLDER}/{fig_name}.png', bbox_inches='tight')


## 1.4 Des routines statistiques

Les routines statistiques utilisées dans ce notebook sont regroupés dans cette section :

- format des outputs
- normalité d'une série
- homoscédasticité de groupes pour une série


In [5]:
def format_stat_p(stat, p, name=None):
    ch = f'{name} : ' if name else ''
    ch += f'stat={stat:.3f}, p={p:.3f}'
    if p < 0.001:
        ch += '***'
    elif p < 0.01:
        ch += '**'
    elif p < 0.05:
        ch += '*'
    return ch


### 1.4.1 Des tests de normalité

- S'il y a moins de 50 observations, utilise Shapiro test,
- Sinon, utilise 'NormalTest' fourni par scipy qui est basé sur D’Agostino et Pearson.

Un alternatif est d'utiliser le test de Kolmogorov-Smirnov avec comparaison à la distribution normal ('goodness of fit')


In [6]:
from scipy.stats import shapiro, kstest


def test_normality(series: pd.Series, alpha=0.05):
    s = series.astype(float).dropna()
    if len(s) < 3:
        # il faut au moins 3 points
        return
    if len(s) < 50:
        stat, p = shapiro(s)
        ch = format_stat_p(stat, p, name=f'Shapiro [{s.name}]')
    else:
        stat, p = kstest(s, cdf='norm')
        ch = format_stat_p(stat, p, name=f'Kolmogorov-Smirnov [{s.name}]')
        # stat, p = normaltest(s)
        # ch = format_stat_p(stat, p, name=f'NormalTest [{s.name}]')
    if p > alpha:
        choix = 'accept H0: probablement Gaussian'
    else:
        choix = 'reject H0: probablement pas Gaussian'
    return p, ch, choix


### 1.4.2 Tests d'égalité de variance des groupes (homoscédasticité)

- si les groupes ont toutes une distribution normale, utilise Bartlett (test paramétrique)
- sinon, utilise Levene (test non paramétrique)


In [7]:
def group_by_groups(df: pd.DataFrame, y_var: str, groups_var: str) -> list:
    """Créer une liste de groupes de y_var pour tests statistiques entre groupes"""
    data = df[[y_var, groups_var]].dropna()
    # certain tests n'accepte pas type int
    data[y_var] = data[y_var].astype(float)
    groups = []
    group_names = data[groups_var].unique()
    group_names.sort()
    for group_name in group_names:
        groups.append(data[data[groups_var] == group_name]
                      [y_var].rename(group_name))
    return groups


def get_grp_size(groups: list) -> str:
    group_sizes = [len(g) for g in groups]
    return f'(n={np.sum(group_sizes)}), {group_sizes}'


In [8]:

from scipy.stats import bartlett, levene


def test_homoscedascity(groups: list, normality=False, alpha=0.05):
    print(f'test_homoscedascity groups : {get_grp_size(groups)}')
    if normality:
        stat, p = bartlett(*groups)
        ch = format_stat_p(stat, p, 'Bartlett homoscédasticité')
    else:
        stat, p = levene(*groups)
        ch = format_stat_p(stat, p, 'Levene homoscédasticité')
    if p > alpha:
        choix = 'accept H0: les groupes ont un écart type similaire'
    else:
        choix = 'reject H0: les groupes ont des écart types différents'
    return p, ch, choix


# 2. Import et nettoyage des données

Les données de consommation sont à télécharger à [cette adresse](https://www.kaggle.com/city-of-seattle/sea-building-energy-benchmarking#2015-building-energy-benchmarking.csv).


## 2.1 Description des données (metadata)

- les descriptions des champs sont fournies par les fichiers metadata (format json)
- conversion en CSV pour faciliter la lecture


In [9]:
import os
import json


def create_data_dict_csv_from_json(file: str, outfile: str) -> pd.DataFrame:
    if os.path.exists(outfile):
        return pd.read_csv(outfile).iloc[:, -3:]
    data = json.loads(open(file, "r").read())
    df = pd.json_normalize([data], record_path='columns', max_level=0)
    df = df[['name', 'dataTypeName', 'description']]
    df.to_csv(outfile)
    return df


meta2015 = 'data/raw/socrata_metadata_2015-building-energy-benchmarking.json'
meta2016 = 'data/raw/socrata_metadata_2016-building-energy-benchmarking.json'

if not os.path.exists('data/out'):
    os.makedirs('data/out')
out2015 = 'data/out/datadict2015.csv'
out2016 = 'data/out/datadict2016.csv'

dict2015 = create_data_dict_csv_from_json(meta2015, out2015)
dict2016 = create_data_dict_csv_from_json(meta2016, out2016)
print(f'2015 : il y a {len(dict2015)} colonnes')
print(f'2016 : il y a {len(dict2016)} colonnes')


2015 : il y a 47 colonnes
2016 : il y a 46 colonnes


### 2.1.1 Les variables cibles (targets) à prédire

En regardent les descriptions des champs (les fichiers CSV), on voit que les champs à prédire sont :

- `SiteEnergyUse(kBtu)` = **consommation totale d’énergie**
- `TotalGHGEmissions` = **les émissions de CO2 (equivalence)**

Ces 2 champs sont probablement fortement corrélés avec `PropertyGFABuilding(s)` = le surface totale intérieure du bâtiment : On attend qu'un gros bâtiment aura plus de consommation d'énergie qu'un petit bâtiment, pour la même type d'usage.

#### 2.1.1.1 Les variables alternatifs à prédire

Sans transformation des residues d'erreur, la modèle risque de mettre trop de poids pour les gros bâtiments. Une façon de réduire le poids des grands bâtiments sera de prédire :

- `SiteEUI(kBtu/sf)` = **consommation totale d’énergie** divisé par surface totale intérieure du bâtiment
- `GHGEmissionsIntensity` = **les émissions de CO2 (equivalence)** divisé par surface totale intérieure du bâtiment

Puis, multiplié ces previsions par la surface totale du bâtiment pour arriver à la prévision de consommation totale d'énergie.

On évaluera les effets pendant la modélisation


In [10]:
target_cols = ['SiteEnergyUse(kBtu)', 'SiteEUI(kBtu/sf)',
               'TotalGHGEmissions', 'GHGEmissionsIntensity']

dict2016[dict2016['name'].isin(target_cols)]


,name,dataTypeName,description
29,SiteEUI(kBtu/sf),number,"Site Energy Use Intensity (EUI) is a property's Site Energy Use divided by its gross floor area. Site Energy Use is the annual amount of all the energy consumed by the property on-site, as reported on utility bills. Site EUI is measured in thousands of British thermal units (kBtu) per square foot."
33,SiteEnergyUse(kBtu),number,The annual amount of energy consumed by the property from all sources of energy.
44,TotalGHGEmissions,text,"The total amount of greenhouse gas emissions, including carbon dioxide, methane, and nitrous oxide gases released into the atmosphere as a result of energy consumption at the property, measured in metric tons of carbon dioxide equivalent. This calculation uses a GHG emissions factor from Seattle CIty Light's portfolio of generating resources. This uses Seattle City Light's 2015 emissions factor of 52.44 lbs CO2e/MWh until the 2016 factor is available. Enwave steam factor = 170.17 lbs CO2e/MMBtu. Gas factor sourced from EPA Portfolio Manager = 53.11 kg CO2e/MBtu."
45,GHGEmissionsIntensity,text,"Total Greenhouse Gas Emissions divided by property's gross floor area, measured in kilograms of carbon dioxide equivalent per square foot. This calculation uses a GHG emissions factor from Seattle City Light's portfolio of generating resources"


### Est-ce que les données de 2015 et 2016 sont compatibles ?


In [11]:
cols_unique2015 = set(list(dict2015['name'])) - set(list(dict2016['name']))
cols_unique2016 = set(list(dict2016['name'])) - set(list(dict2015['name']))
print(f'colonnes unique à année 2015 : {sorted(cols_unique2015)}')
print(f'colonnes unique à année 2016 : {sorted(cols_unique2016)}')


colonnes unique à année 2015 : ['2010 Census Tracts', 'City Council Districts', 'Comment', 'GHGEmissions(MetricTonsCO2e)', 'GHGEmissionsIntensity(kgCO2e/ft2)', 'Location', 'OtherFuelUse(kBtu)', 'SPD Beats', 'Seattle Police Department Micro Community Policing Plan Areas', 'Zip Codes']
colonnes unique à année 2016 : ['Address', 'City', 'Comments', 'GHGEmissionsIntensity', 'Latitude', 'Longitude', 'State', 'TotalGHGEmissions', 'ZipCode']


# 3. Analyse Exploratoire


# 4. Feature Engineering


# 5. Enregistre les données nettoyées
